<a href="https://colab.research.google.com/github/huan/tensorflow-handbook-swift/blob/master/tensorflow-handbook-swift-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Swift MNIST Example

Learn more from Github: https://github.com/huan/tensorflow-handbook-swift


## MNIST Dataset

<https://github.com/huan/swift-MNIST/>

In [1]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/huan/swift-MNIST", from: "0.4.0")' MNIST

Installing packages:
	.package(url: "https://github.com/huan/swift-MNIST", from: "0.4.0")
		MNIST
With SwiftPM flags: []
Working in: /tmp/tmpdu69j2gk/swift-install
Fetching https://github.com/huan/swift-MNIST
Cloning https://github.com/huan/swift-MNIST
Resolving https://github.com/huan/swift-MNIST at 0.4.0
[1/2] Compiling MNIST MNIST.swift
[2/3] Merging module MNIST
[3/4] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[4/5] Merging module jupyterInstalledPackages
[5/5] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


## Define a Simple MLP Model

In [0]:
import TensorFlow
import Python
import Foundation

struct MLP: Layer {
  typealias Input = Tensor<Float>
  typealias Output = Tensor<Float>

  var flatten = Flatten<Float>()
  var dense = Dense<Float>(inputSize: 784, outputSize: 10)
  
  @differentiable
  public func callAsFunction(_ input: Input) -> Output {
    var x = input
    x = flatten(x)
    x = dense(x)
    return x
  }  
}

var model = MLP()
let optimizer = Adam(for: model)

## Training

In [3]:
import MNIST

let mnist = MNIST()
let ((trainImages, trainLabels), (testImages, testLabels)) = mnist.loadData()

let imageBatch = Dataset(elements: trainImages).batched(32)
let labelBatch = Dataset(elements: trainLabels).batched(32)

for (X, y) in zip(imageBatch, labelBatch) {
  // Caculate the gradient
  // let (_loss, grads) = valueWithGradient(at: model) { model -> Tensor<Float> in
  let grads = gradient(at: model) { model -> Tensor<Float> in
    let logits = model(X)
    return softmaxCrossEntropy(logits: logits, labels: y)
  }

  // Update parameters by optimizer
  optimizer.update(&model.allDifferentiableVariables, along: grads)    
}

let logits = model(testImages)
let acc = mnist.getAccuracy(y: testLabels, logits: logits)

print("Test Accuracy: \(acc)" )

Reading data.
Constructing data tensors.
Test Accuracy: 0.9125


- Credit: This example is inspired from [A set of notebooks explaining swift for tensorflow optimized to run in Google Collaboratory.](https://github.com/zaidalyafeai/Swift4TF)
- License [Apache-2.0](https://github.com/tensorflow/swift-models/blob/stable/LICENSE)